In [ ]:
%matplotlib inline
import h5py
from tomo_recon_20210811 import rotcen_test2
import numpy as np
import matplotlib.pyplot as plt
import tomopy

## Load the file

In [ ]:
fn = r'fly_scan_id_30543.h5'

## 1. Visualize the projection

In [ ]:
from tiled.client import from_uri
c = from_uri("https://tiled-demo.blueskyproject.io")
c["um2023"]

In [ ]:
f = h5py.File(fn, 'r')
prj = np.array(f['img_tomo'])

fig, ax = plt.subplots()

ax.imshow(prj[prj.shape[0] // 2])
angle_ = f['angle'][prj.shape[0] // 2]
ax.set_title(rf'Projection at: {angle_:0.2f} degree')

plt.show()

## 2. Tomography reconstruction on sinogram
    - rotation center test
    - recon by tomopy
    - visualization

In [ ]:
# rotation center test
# need to pick up rotation center manually
# In this case, it is 651
rotcen_test2(fn, 630, 690, 41, denoise_flag=2, dark_scale=4)

In [ ]:
# reconstruction for one sinogram


sli = 540   # specify the slice to recon
rotation_center = 651   # rotation center obtained by rotcen_test2 from the cell above

# projection normalization
img_tomo = np.array(f["img_tomo"][:, sli : sli+1, :])
img_bkg = np.array(f["img_bkg_avg"][:, sli : sli+1, :])
img_dark = np.array(f["img_dark_avg"][:, sli : sli+1, :])
prj = (img_tomo - img_dark) / (img_bkg - img_dark)  # normalize the projection
prj_norm = -np.log(prj) # -log

# data cleaning
prj_norm[np.isnan(prj_norm)] = 0
prj_norm[np.isinf(prj_norm)] = 0
prj_norm[prj_norm < 0] = 0

prj_norm = tomopy.prep.stripe.remove_stripe_fw(
    prj_norm, level=9, wname="db5", sigma=1, pad=True
)

# load the angles
tomo_angle = np.array(f["angle"]) 
theta = tomo_angle / 180.0 * np.pi

# recon by tomopy
img = tomopy.recon(
prj_norm[:, :],
theta,
center=rotation_center,
algorithm="gridrec",
filter_name='None'
)

In [ ]:
# visualize the sinogram and the recon result
%matplotlib inline

fig, ax = plt.subplots(1, 2, figsize=(12, 8))

ax[0].imshow(img_tomo[:, 0, :])
ax[0].set_title('Sinogram')

ax[1].imshow(img[0, ...])
ax[1].set_title('Recon')

## Citation
1. "Formation of three-dimensional bicontinuous structures via molten salt dealloying studied in real-time by in situ synchrotron X-ray nano-tomography"
Xiaoyang Liu, Arthur Ronne*, Lin-Chieh Yu, Yang Liu, Mingyuan Ge, Cheng-Hung Lin, Bobby Layne, Phillip Halstenberg, Dmitry S. Maltsev, Alexander S. Ivanov, Stephen Antonelli, Sheng Dai, Wah-Keat Lee, Shannon M. Mahurin, Anatoly I. Frenkel, James F. Wishart, Xianghui Xiao & Yu-chen Karen Chen-Wiegart*
Nature Communications (2021), DOI: 10.1038/s41467-021-23598-8

2. "Temperature-Dependent Morphological Evolution during Corrosion of the Ni-20Cr Alloy in Molten Salt Revealed by Multiscale Imaging"
Xiaoyang Liu, Kaustubh Bawane, Xiaoyin Zheng, Mingyuan Ge, Phillip Halstenberg, Dmitry S. Maltsev, Alexander S. Ivanov, Sheng Dai, Xianghui Xiao, Wah-Keat Lee, Lingfeng He, and Yu-chen Karen Chen-Wiegart*
ACS Applied Materials & Interfaces (2023), DOI: 10.1021/acsami.2c23207

3. Mingyuan Ge, Wah-Keat Lee. PyXAS – an open-source package for 2D X-ray near-edge spectroscopy analysis. Journal of Synchrotron Radiation, 27, 567 (2020)